In [7]:
# libraries
import os
import requests
import threading
import pandas as pd
from bs4 import BeautifulSoup as bs4
from requests.adapters import HTTPAdapter

# from Project
from GetSheetStatement import Constractv3


class MOPS(object):

    def __init__(self, sheet: str, sid: str, year: int, season: int):
        #super().__init__()
        self.sheet = Constractv3.FINANCIAL_STATMENT[sheet]
        self.url = Constractv3.MOPS_URL.format(sheet=self.sheet)
        Constractv3.HEADERS['Referer'].format(sheet=self.sheet)
        self.header = Constractv3.HEADERS
        self.payload = Constractv3.PAYLOAD
        self.payload ["co_id"] = sid
        self.payload["year"] = int(year)
        self.payload["season"] = f"0{season}"
        self.file_path = Constractv3.FILE_PATH.format(
            sheet=sheet, sid=self.payload["co_id"],year=self.payload["year"] + 1911, season=self.payload["season"])
        if not os.path.exists(sheet):
            os.mkdir(sheet)

        print(f"INIT Thread: {sheet}\nURL: {self.url}\nFILE_PATH: {self.file_path}\n")
        pass

    def run(self):
        # set session retry option
        web_ss = requests.session()
        ss_adapter = HTTPAdapter(max_retries=3)
        web_ss.mount("https://", adapter=ss_adapter)
        try:
            # start get html
          
            res = web_ss.post(url=self.url, headers=self.header, data=self.payload, timeout=3)
            
            # check iff: Response OK
            if res.status_code == 200:
                soup = bs4(res.text, "html.parser")
                tables = soup.select('table.hasBorder')
                #print(tables)
                return tables
            
        except Exception as e:
            print(f"Get HTML FAIL: {e}", end="\n\n")


#B is 資產負債表
#sheet = 'B'

#P is 損益表
#sheet= 'P' 

#C is 現金流量表
#sheet= 'C'

#S is 股東權益表
sheet= 'B' 


sid = '1101'
year = 108
season= 4

MOPS=MOPS(sheet, sid, year, season)
result=MOPS.run()

#print(result[0])


even = result[0].select('td.even')
odd = result[0].select('td.odd')


#for i, v in enumerate(even):
#    print(i, v.text)
    
for i, v in enumerate(odd):
    print(i, v.text)


# %k
# B: 1, 2, 3 = 5
# B: 4 = 7
# P: 1, 2, 3, 4 = 5
# C = 3

if sheet=='C':
    k = 3
else:
    if season==4 or sheet == 'P':
        k=5
    else:
        k=7

INIT Thread: B
URL: https://mops.twse.com.tw/mops/web/ajax_t164sb03
FILE_PATH: ./B/1101-2019-04.json

0 　　　現金及約當現金
1                      53,572,620
2                              14.58
3                      48,507,889
4                              14.10
5 　　　透過其他綜合損益按公允價值衡量之金融資產－流動
6                       7,180,745
7                               1.95
8                       6,334,259
9                               1.84
10 　　　應收票據淨額
11                      31,258,167
12                               8.51
13                      29,748,544
14                               8.65
15 　　　應收帳款淨額
16                       8,838,553
17                               2.41
18                       8,216,174
19                               2.39
20 　　　其他應收款淨額
21                         913,545
22                               0.25
23                         955,814
24                               0.28
25 　　　其他流動資產
26                         910,672
27                               0.25
28      

In [8]:
columns_odd=[]

for i, v in enumerate(even):
    if i% k ==0 :
        columns_odd.append(v.text.strip())
#print(columns_odd , len(columns_odd))

columns_even=[]
for i, v in enumerate(odd):
    if i% k ==0 :
        columns_even.append(v.text.strip())
        
#print(columns_even , len(columns_even))

columns=[]
for _o, _e in zip(columns_odd, columns_even):
    columns.append(_o)
    columns.append(_e)
    
print(columns, len(columns))

['流動資產', '現金及約當現金', '透過損益按公允價值衡量之金融資產－流動', '透過其他綜合損益按公允價值衡量之金融資產－流動', '按攤銷後成本衡量之金融資產－流動', '應收票據淨額', '應收帳款－關係人淨額', '應收帳款淨額', '存貨', '其他應收款淨額', '預付款項', '其他流動資產', '流動資產合計', '非流動資產', '按攤銷後成本衡量之金融資產－非流動', '透過其他綜合損益按公允價值衡量之金融資產－非流動', '不動產、廠房及設備', '採用權益法之投資', '使用權資產', '投資性不動產淨額', '無形資產', '其他非流動資產', '非流動資產合計', '資產總額', '流動負債', '短期借款', '應付短期票券', '透過損益按公允價值衡量之金融負債－流動', '合約負債－流動', '應付帳款', '租賃負債－流動', '其他應付款', '流動負債合計', '本期所得稅負債', '應付公司債', '其他流動負債', '租賃負債－非流動', '非流動負債', '非流動負債合計', '長期借款', '歸屬於母公司業主之權益', '遞延所得稅負債', '普通股股本', '其他非流動負債', '資本公積', '負債總額', '資本公積合計', '股本', '其他權益', '特別股股本', '其他權益合計', '股本合計', '歸屬於母公司業主之權益合計', '保留盈餘', '非控制權益', '保留盈餘合計', '負債及權益總計', '庫藏股票', '預收股款（權益項下）之約當發行股數（單位：股）', '權益總額'] 60


In [10]:
content_odd=[]
for i, v in enumerate(even):
    if i% k ==0 :
        content_odd.append(even[i+1].text.strip().replace(',',''))   
#print(content_odd)

content_even=[]
for i, v in enumerate(odd):
    if i% k ==0 :
        content_even.append(odd[i+1].text.strip().replace(',',''))   
#print(content_even)

content=[]
for _o, _e in zip(content_odd, content_even):
    content.append(_o)
    content.append(_e)
    
#print(content, len(content))

Newcolumns=['sID','Quarter']
Newcontent=[sid, season]
Newcolumns.extend(columns)
Newcontent.extend(content)


df=pd.DataFrame(columns=Newcolumns)

df.loc[1]=Newcontent


#json = df.to_json(orient='records')

df.to_json(MOPS.file_path, orient='records')

df

,sID,Quarter,流動資產,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,應收票據淨額,應收帳款－關係人淨額,應收帳款淨額,...,其他權益合計,股本合計,歸屬於母公司業主之權益合計,保留盈餘,非控制權益,保留盈餘合計,負債及權益總計,庫藏股票,預收股款（權益項下）之約當發行股數（單位：股）,權益總額
1,1101,4,,53572620,502005,7180745,4745911,31258167,300000,8838553,...,23734855,56656192,193684068,,14777799,65626033,367482385,-348959,0,208461867
